In [1]:
import pandas as pd
import os
from datetime import date, datetime, timedelta
import numpy as np
import seaborn as sns

C:\Users\marie\AppData\Local\Temp\ipykernel_11556\1657975687.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
def interval_seconds(interval_str):
    if len(interval_str.split(" d ")) > 1:
        days = int(interval_str.split(" d ")[0])
        times = interval_str.split(" d ")[1].split(":")
    else:
        days = 0
        times = interval_str.split(" d ")[-1].split(":")
    hours = int(times[0])
    minutes = int(times[1])
    seconds = int(times[2])
    interval = timedelta(days = days, hours = hours, minutes = minutes, seconds = seconds).total_seconds()
    return interval

In [3]:
dirpath = os.getcwd()
data_path = os.path.join(dirpath, "..", "data")

df = pd.read_csv(os.path.join(data_path, "delpro_visit_lactation.csv")) # , header = 0,  names = names, dtype = dtypes)
df = df[df['date'].notna()]
df.date = [datetime.strptime(day, "%Y-%m-%d") for day in df.date]
df["interval_sec"] = [interval_seconds(rec) for rec in df.interval]

agg_df = df[["animal_number", "interval_sec", "yield", "yield_lf", "yield_lr", "yield_rr", "yield_rf"]]\
    .groupby("animal_number")\
    .agg(yield_lf_visit_sum = ('yield_lf', 'sum'),\
         yield_lr_visit_sum = ('yield_lr', 'sum'),\
         yield_rr_visit_sum = ('yield_rr', 'sum'),\
         yield_rf_visit_sum = ('yield_rf', 'sum'),\
         interval_visit_mean = ("interval_sec",'mean')).reset_index()

agg_df.to_csv(os.path.join(data_path, "yield_lactation.csv"), index = False)